In [1]:
import subprocess
import shlex

def run_command(command):
    print("Launch " + command)
    process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [2]:
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt 

def get_df_process(host):
    host="vm-vmonly-db-from1"
    port=5432
    login="bonitauser"
    password="migrationTestDb_18"
    dbname="bonitadb"
    conn = psycopg2.connect("host = '{0}' dbname = '{1}' user = '{2}' password = '{3}' port = {4}".
             format(host, dbname, login, password, port))
    sql = "select api_id.sourceobjectid as id, to_timestamp(api_begin.startdate::float/1000) as start, to_timestamp(api_end.enddate::float/1000) as end, api_end.enddate - api_begin.startdate as duration \n"\
    "from ( \n"\
    "	select distinct sourceobjectid \n"\
    "	from arch_process_instance api) as api_id \n"\
    "join arch_process_instance api_begin on api_id.sourceobjectid = api_begin.sourceobjectid and api_begin.stateid = 0 \n"\
    "left join arch_process_instance api_end on api_id.sourceobjectid = api_end.sourceobjectid and api_end.stateid = 6 \n"\
    "order by id"
    cur = conn.cursor()
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns=["id", "start", "end", "duration"])
    return df

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Initialize result name, Start Faban and reset Docker

In [3]:
# 

now=datetime.datetime.now(tzlocal())
strnow = now.strftime("%Y%m%d%H%M%S")
migration_result_name = "xp2_migr_{0}".format(strnow)
print ("Migration result name : " + migration_result_name)
with cd("./test_executor/faban/master/bin"):
    run_command("./shutdown.sh")
    run_command("./startup.sh")


run_command("./reset_docker.sh")
    

Migration result name : xp2_migr_20180615132957
Launch ./shutdown.sh
Launch ./startup.sh
Starting Faban Server
Please point your browser to http://127.0.0.1:9980/
Launch ./reset_docker.sh
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_bonita-from
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_bonita-to
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_db-from
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_db-ref
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_db-to
Removing service 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_start_dependencies
Removing network 4xlujqozyr1PobRVrErvLAwu5hgkbGfh_default


0

### Provision and test for various numbers of processes

In [26]:
run_command("./reset_docker.sh")

Launch ./reset_docker.sh
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_bonita-from
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_bonita-to
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_db-from
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_db-ref
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_db-to
Removing service tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_start_dependencies
Removing network tI7SMd7l8t2SWr2YYyxW0MpOn6h8SLgV_default


0

In [27]:
# provision and test

file = open("{0}.csv".format(migration_result_name), "w")
file.write("ts,nb,duration_disable,duration_move,duration_enable,duration_total\n")
file.close()

run_command("./reset_docker.sh")
dur = 30
#run_command("./launch_agents.sh http://vm-vmonly-bpms-from1:8080/bonita")
with cd ("./test_executor"):
    run_command("./benchflow_onlytest.sh generate_test 5 TestHumanTask-1.0.bar 5 {0} 0 tenant1".format(dur))

    run_command("./benchflow_onlytest.sh execute_tests deploy")

Launch ./reset_docker.sh
"docker stack rm" requires at least 1 argument.
See 'docker stack rm --help'.

Usage:  docker stack rm STACK [STACK...] [flags]

Remove one or more stacks
Launch ./benchflow_onlytest.sh generate_test 5 TestHumanTask-1.0.bar 5 30 0 tenant1
The generated test ID is 2zABTYlIQhVI6CUzfW1Pr7Tu2A8bUgfu
Compiling the Faban driver
Buildfile: /home/ubuntu/scripts_gr/test_executor/drivers/2zABTYlIQhVI6CUzfW1Pr7Tu2A8bUgfu/build.xml

init:
[mkdir] Created dir: /home/ubuntu/scripts_gr/test_executor/drivers/2zABTYlIQhVI6CUzfW1Pr7Tu2A8bUgfu/build/classes

compile:
[javac] Compiling 1 source file to /home/ubuntu/scripts_gr/test_executor/drivers/2zABTYlIQhVI6CUzfW1Pr7Tu2A8bUgfu/build/classes
[javac] warning: [options] bootstrap class path not set in conjunction with -source 1.7
[javac] 1 warning

bench.jar:
[mkdir] Created dir: /home/ubuntu/scripts_gr/test_executor/drivers/2zABTYlIQhVI6CUzfW1Pr7Tu2A8bUgfu/build/lib
[jar] Building jar: /home/ubuntu/scripts_gr/test_executor/driver

In [ ]:
res1 = run_command("./migrate.sh tenant1 {0}".format(migration_result_name))
if (res1 !=0):
    print("Error in first migration.. skipping")
df_pre = get_df_process("vm-vmonly-db-from1")
#df_pre.plot(x="start", y="duration", style=".")



In [ ]:
date_postmigr = datetime.datetime.now(tzlocal())

run_command("./launch_agents.sh http://vm-vmonly-bpms-to1:8081/bonita")
with cd ("./test_executor"):
    run_command("./benchflow_onlytest.sh generate_test 5 TestHumanTask-1.0.bar 5 {0} 0 tenant1 config2.properties".format(dur))

    run_command("./benchflow_onlytest.sh execute_tests")

df = get_df_process("vm-vmonly-db-from1")

df_pre = df[df["start"] < date_postmigr]
df_post = df[df["start"] >= date_postmigr]

df_pre.to_csv("{0}_pre_{1}.csv".format(migration_result_name ,i))
df_post.to_csv("{0}_post_{1}.csv".format(migration_result_name ,i))    

df_pre["start"] = df_pre["start"] - min(df_pre["start"])
df_post["start"] = df_post["start"] - min(df_post["start"])

df_pre = df_pre.set_index(pd.DatetimeIndex(df_pre["start"]))   
df_post=df_post.set_index(pd.DatetimeIndex(df_post["start"]))
fig,ax = plt.subplots()
#
#

df_pre.resample("1s", how="mean").plot(y="duration", style="ro",ax=ax)
df_post.resample("1s", how="mean").plot( y="duration", style="bx",ax=ax)